In [1]:
from os import listdir
from os.path import isfile, join
import nltk
import re
import conllu
import regex

#### Função para obter as introduçãos de todos os arquivos de um determinado diretório

In [2]:
def get_introductions(dir):
    documents = []
    introductions = []
    for infile in listdir(dir):
        if infile[-5:] != '.text':
            continue
            
        with open(join(dir, infile), 'r') as file:
            text = file.read()

        regex_head = re.search('---\n[\S\n ]*---\n', text)
        head = regex_head.group()
        natureza = re.search('natureza: [\S ]*\n', head).group()
        if natureza != 'natureza: biográfico\n':
            continue

        paragraphs = text[regex_head.end():]
        delimiter = re.search('\n\n', paragraphs)
        if delimiter:
            first_paragraph = paragraphs[:delimiter.start()]
        else:
            first_paragraph = paragraphs
        first_paragraph = re.sub('\n', ' ', first_paragraph)
        first_paragraph = re.sub(' +', ' ', first_paragraph)
        if first_paragraph in ['', ' ']:
            continue
            
        introduction = nltk.tokenize.sent_tokenize(first_paragraph)[0]
        # trocar filha por filho melhora os resultados do parser
        introduction = re.sub('filha', 'filho', introduction)
        introduction = re.sub('«', '', introduction)
        introduction = re.sub('»', '', introduction)
        introduction = re.sub('^ ', '', introduction)
        introduction = re.sub(' $', '', introduction)
        
        documents.append(infile[:-5])
        introductions.append(introduction)
    
    return documents, introductions

#### Nome dos documentos e as introduções dado o diretório dhbb/text

In [3]:
documents, introductions = get_introductions('dhbb/text')

#### Salvar introduçãos no arquivo introductions.text

In [4]:
with open('introductions.text', 'w') as file:
    file.write('\n\n'.join(introductions))

#### Rodando o parser udpipe/src/udpipe dado o modelo portuguese-gsd-ud-2.5-191206.udpipe e o arquivo introductions.text. A saída é o arquivo introductions.conllu

In [5]:
!./udpipe/src/udpipe --tokenize --tokenizer=presegmented --tag --parse portuguese-gsd-ud-2.5-191206.udpipe introductions.text --outfile introductions.conllu

Loading UDPipe model: done.


#### Abrindo o arquivo introductions.conllu e armazendo as sentenças

In [6]:
with open('introductions.conllu', 'r') as file:
    conllu_text = file.read()
sentences = conllu.parse(conllu_text)

#### Funções para obter o ramo da árvore da data de nascimento. is_date_tree1 é mais certo

In [7]:
def is_date_tree1(tree):
    return (tree.token['lemma'] == 'dia'
            or tree.token['lemma'] == 'mês'
            or tree.token['lemma'] == 'ano')

def is_date_tree2(tree):
    return ((tree.token['deprel'] == 'nmod' 
             and tree.token['upos'] == 'NUM') # dia / ano
            or (tree.token['deprel'] == 'nmod' 
                and tree.token['upos'] == 'PROPN')) # mês

#### Funções para obter o ramo da árvore do local de nascimento. is_local_tree1 é mais certo

In [8]:
def is_local_tree1(tree):
    return (tree.token['deprel'] == 'nmod' 
            and tree.token['upos'] == 'PROPN') # nome do local

def is_local_tree2(tree):
    return (tree.token['deprel'] == 'nmod' 
            and tree.token['upos'] == 'NOUN') # tipo do local

#### Funções para obter o ramo da árvore dos pais. is_parent_tree1 é mais certo

In [9]:
def is_parent_tree1(tree):
    return tree.token['lemma'] == 'filho'

def is_parent_tree2(tree):
    return (tree.token['deprel'] == 'conj'
            and tree.token['upos'] == 'PROPN') # nome

#### Função para dado ramos e alguma das funções anteriores, retornar o ramo correspondente

In [10]:
def get_part_branch(branches, func):
    for i, branch in enumerate(branches):
        if func(branch):
            branches.pop(i)
            return branches[i:], branch
    return branches, None

#### Função para dado uma árvore, retornar todos os ramos desejados

In [11]:
def get_parts(tree):
    branches = tree.children[:-1].copy()
    
    name_tree = branches.pop(0)
    
    branches_local , local_tree  = get_part_branch(branches      , is_local_tree1)
    branches_date  , date_tree   = get_part_branch(branches_local, is_date_tree1)
    branches_parent, parent_tree = get_part_branch(branches_date , is_parent_tree1)
    
    if not local_tree:
        branches_local , local_tree   = get_part_branch(branches      , is_local_tree2)
        
    if not date_tree:
        branches_date  , date_tree    = get_part_branch(branches_local, is_date_tree2)
        
    if not parent_tree:
        branches_parent, parent_tree  = get_part_branch(branches_date , is_parent_tree2)
        
    if parent_tree:
        _              , parent2_tree = get_part_branch(branches_parent, is_parent_tree2)
    else:
        parent2_tree = None
        
    return name_tree, local_tree, date_tree, parent_tree, parent2_tree

#### Função para retornar os tokens de um ramo

In [12]:
def _get_tokens(tree):
    tokens = [tree.token]
    for child in tree.children:
        tokens = tokens + _get_tokens(child)
        
    return tokens

def get_tokens(tree):
    tokens = _get_tokens(tree)
    return sorted(tokens, key=lambda token: token['id'])

#### Função para retornar o texto de um ramo

In [13]:
def get_text(tree):
    tokens = get_tokens(tree)
    return ' '.join([token['form'] for token in tokens])

#### Função para obter somente nome dado o ramo

In [14]:
def get_name(name_tree):
    text = get_text(name_tree)
    rex = regex.compile(r'[[:upper:]][\w ]*')
    match = rex.search(text)
    if match:
        return match.group(0)

#### Função para obter somente o local de nascimento dado o ramo

In [15]:
def get_local(local_tree):
    if local_tree:
        text = get_text(local_tree)
        text = re.sub('\( ', '(', text)
        text = re.sub(' \)', ')', text)
        rex = regex.compile(r'([[:upper:]][\w ]* \([A-Z][A-Z]\))|([[:upper:]][\w ]*$)')
        match = rex.search(text)
        if match:
            return match.group(0)

#### Função para obter somente a data de nascimento normalizada dado o ramo

In [16]:
meses = [('01', 'janeiro'),
         ('02', 'fevereiro'),
         ('03', 'março'),
         ('04', 'abril'),
         ('05', 'maio'),
         ('06', 'junho'), 
         ('07', 'julho'),
         ('08', 'agosto'),
         ('09', 'setembro'),
         ('10', 'outubro'),
         ('11', 'novembro'),
         ('12', 'dezembro')]

def get_date(datetree):
    if datetree:
        text = get_text(datetree).lower()
        text = re.sub(' º', '', text)
        for num, mes in meses:
            text = re.sub(mes, num, text)
        rex = regex.compile(r'(\d{1,2} de \d\d de \d\d\d\d)|(\d\d de \d\d\d\d)|(\d\d\d\d)')
        match = rex.search(text)
        if match:
            text = match.group(0)
            text = re.sub(' de ', '/', text)
            return text

#### Função para obter somente os pais dado os ramos

In [17]:
def get_parents(parent_tree, parent2_tree):
    if parent_tree:
        text = get_text(parent_tree)
        if parent2_tree:
            text = text + ' ' + get_text(parent2_tree)
        texts = text.split(' e ')
        text1 = texts[0]
        if len(texts) > 1:
            text2 = texts[1]
        else:
            text2 = ''
        
        rex = regex.compile(r'[[:upper:]][\w ]*')
        
        match1 = rex.search(text1)
        if match1:
            text1 = match1.group(0)
        else:
            text1 = None
            
        match2 = rex.search(text2)
        if match2:
            text2 = match2.group(0)
        else:
            text2 = None
            
        return text1, text2
    
    return None, None

#### Função para escrever no arquivo as informações desejadas dado as sentenças. Roda os processos anteriores

In [18]:
def write_data(outfile, documents, sentences):
    with open(outfile, 'w') as file:
        for document, sentence in zip(documents, sentences):
            file.write('---\n')
            tree = sentence.to_tree()
            name_tree, local_tree, date_tree, parent_tree, parent2_tree = get_parts(tree)
            if name_tree:
                name = get_name(name_tree)
                local = get_local(local_tree)
                date = get_date(date_tree)
                parent1, parent2 = get_parents(parent_tree, parent2_tree)

                file.write(f'document   : {document}\n')
                file.write(f'name       : {name}\n')
                if local:
                    file.write(f'birthplace : {local}\n')
                if date:
                    file.write(f'birthdate  : {date}\n')
                if parent1:
                    file.write('parents    :\n')
                    file.write(f'  - {parent1}\n')
                    if parent2:
                        file.write(f'  - {parent2}\n')

#### Executando a função para sentenças e o arquivo de saída data.yaml

In [19]:
write_data('data.yaml', documents, sentences)